In [110]:
import pandas as pd
import csv

In [109]:
freqActors = dict()
file = open("data/principals.tsv")
reader = csv.reader(file)
lines= len(list(reader))
#df = pd.read_csv('data/principals.tsv',sep='\t')

KeyboardInterrupt: 

In [111]:
count = 0
for chunk in pd.read_csv('data/principals.tsv',sep='\t',chunksize=1000000):
    df = chunk
    print("ok")
    actors = df[df['category'] == 'actor']
    baskets = actors['tconst'].unique()
    print(len(baskets))
    for basket in baskets:
        grouping = actors[actors['tconst']== basket]
        for actor in grouping['nconst']:
            #print(actor)
            if actor in freqActors.keys():
                freqActors[actor] +=1
            else:
                freqActors[actor] = 1
    if count%1000 == 0:
        print(count)
    break

ok
102886


KeyboardInterrupt: 

{'nm0443482': 1, 'nm0653042': 1, 'nm0179163': 1, 'nm0183947': 1, 'nm0653028': 1, 'nm0183823': 1, 'nm1309758': 1, 'nm3692297': 1, 'nm0166380': 1, 'nm0244989': 1, 'nm3691272': 1, 'nm3692071': 1, 'nm2350007': 1, 'nm0525907': 1, 'nm1151424': 1, 'nm2354154': 1, 'nm0420198': 9, 'nm0617588': 28, 'nm0525908': 1, 'nm6170115': 1, 'nm0832461': 1, 'nm0577734': 1, 'nm0168621': 8, 'nm0024876': 1, 'nm0278321': 1, 'nm0005717': 1, 'nm0556371': 1, 'nm0563758': 1, 'nm0338379': 3, 'nm0841389': 3, 'nm1024447': 2, 'nm0085865': 6, 'nm1778224': 1, 'nm1778292': 1, 'nm0002504': 1, 'nm0784327': 1, 'nm2259742': 1, 'nm0924893': 1, 'nm0808310': 1, 'nm0335788': 2, 'nm1040375': 1, 'nm1894886': 1, 'nm2940072': 2, 'nm0536252': 1, 'nm0378408': 3, 'nm0804433': 1, 'nm1272063': 1, 'nm1272675': 1, 'nm1012612': 1, 'nm1011210': 1, 'nm1012621': 1, 'nm0675239': 1, 'nm0675260': 1, 'nm0542567': 3, 'nm0509806': 1, 'nm1778194': 1, 'nm0954087': 5, 'nm8709803': 1, 'nm0466448': 1, 'nm5394591': 1, 'nm0924920': 2, 'nm0925513': 1, 'nm002